In [34]:
# For working with and visualizing the data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For training the VAE
import tensorflow as tf

# For creating interactive widgets
import ipywidgets as widgets
from IPython.display import display


In [35]:
face_data_dict = np.load("face_emotion_data (1).npz")
face_data_dict = np.load("face_emotion_data (1).npz")

face_features_u = face_data_dict["X"]
n, p = face_features_u.shape

face_labels_u = face_data_dict["y"]

indices = np.random.permutation(n)

face_features = face_features_u[indices]
face_labels = face_labels_u[indices]

print(face_labels.shape)

print(face_features[0][0], face_labels[0])


(128, 1)
-1.441027187937972 [-1]


In [36]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        
        #reparameterization trick
        #gives the sampled val with the correct variance + shifts the
        #sampled value by the z_mean
        return epsilon * tf.exp(z_log_var * 0.5) + z_mean

In [37]:
def build_vae(num_pixels, num_latent_vars=3):
    
    # Encoder
    encoder_inputs = tf.keras.Input(shape=(num_pixels,))
    x = tf.keras.layers.Dense(512, activation='relu')(encoder_inputs)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    z_mean = tf.keras.layers.Dense(num_latent_vars)(x)
    z_log_var = tf.keras.layers.Dense(num_latent_vars)(z_mean)
    z = Sampling()([z_mean, z_log_var])
    
    encoder = tf.keras.Model(inputs=encoder_inputs, outputs=z)
    
    # Decoder
    decoder_inputs = tf.keras.Input(shape=(num_latent_vars,))
    x = tf.keras.layers.Dense(32, activation='relu')(decoder_inputs)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    reconstruction = tf.keras.layers.Dense(num_pixels, activation='linear')(x)
    
    decoder = tf.keras.Model(inputs=decoder_inputs, outputs=reconstruction)
    
    # Full model
    model_inputs = encoder.input
    model_outputs = decoder(encoder.output)
    
    model = tf.keras.Model(inputs=model_inputs, outputs=model_outputs)
    
    # Compile model for training
    model.compile(
        optimizer='adam',
        loss='mse'
    )
    
    # Return all three models
    return encoder, decoder, model

In [41]:
face_encoder, face_decoder, face_model = build_vae(num_pixels=2304, num_latent_vars=3)
print(face_encoder.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 2304)]       0           []                               
                                                                                                  
 dense_9 (Dense)                (None, 512)          1180160     ['input_3[0][0]']                
                                                                                                  
 dense_10 (Dense)               (None, 128)          65664       ['dense_9[0][0]']                
                                                                                                  
 dense_11 (Dense)               (None, 32)           4128        ['dense_10[0][0]']               
                                                                                            

In [42]:
print(face_decoder.summary())

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense_14 (Dense)            (None, 32)                128       
                                                                 
 dense_15 (Dense)            (None, 128)               4224      
                                                                 
 dense_16 (Dense)            (None, 512)               66048     
                                                                 
 dense_17 (Dense)            (None, 2304)              1181952   
                                                                 
Total params: 1,252,352
Trainable params: 1,252,352
Non-trainable params: 0
_________________________________________________________________
None
